In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
raw_df = (
    spark.read
    .option("sep", " ")
    .option("inferSchema", "true")
    .csv("/Volumes/predictive_cataglog/default/datasets/train_FD001.txt")
)


In [0]:
raw_df.display()

In [0]:

null_columns = [c for c in raw_df.columns if raw_df.filter(col(c).isNotNull()).count() == 0]
raw_df = raw_df.drop(*null_columns)

In [0]:
columns = (
    ["engine_id", "cycle", "op1", "op2", "op3"] +
    [f"s{i}" for i in range(1, 22)]
)

bronze_df = raw_df.toDF(*columns)



In [0]:
bronze_df = bronze_df.withColumn(
    "ingestion_ts",
    current_timestamp()
)

In [0]:
bronze_df.display()

In [0]:
specific_row = bronze_df.filter(bronze_df['engine_id'] == 100).first()
print(specific_row)

In [0]:
bronze_df.groupBy("engine_id") \
    .count() \
    .display()


In [0]:
%sql
create database if not exists predictive_cataglog.bronze_layer;

In [0]:
bronze_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("predictive_cataglog.bronze_layer.bronze_table")

In [0]:
bronze_df.count()

In [0]:
%sql
select * from predictive_cataglog.bronze_layer.bronze_table